Carregando o dataset

In [162]:
import os
import pandas as pd
import numpy as np

dataset = pd.read_pickle(r'C:\Users\mathe\OneDrive\Área de Trabalho\SoccerIA\MathIA_v7\dataset_141cols_europeu.pkl')

nan_counts = dataset.isna().sum()
nan_tot = nan_counts.sum()
print(nan_tot)
print(dataset.shape)

0
(66840, 141)


Importando os dataframes de 2023

In [164]:
import os
import pandas as pd
import numpy as np

# Caminho para a pasta que contém os arquivos .xlsx de 2023
path = r'C:\Users\mathe\OneDrive\Área de Trabalho\SoccerIA\webScrappingOddspedia'

# Dicionário para armazenar os dataframes
dataframes = {}
dfss = []
# Lista todos os arquivos na pasta
files = os.listdir(path)

# Filtra a lista de arquivos para incluir apenas os arquivos .xlsx
xlsx_files = [f for f in files if f.endswith('.xlsx')]

# Carrega cada arquivo .xlsx em um dataframe e armazena no dicionário
for file in xlsx_files:
    full_path = os.path.join(path, file)  # junta o caminho do diretório com o nome do arquivo
    dataframes[file] = pd.read_excel(full_path)  # lê o arquivo .xlsx do caminho completo

    if "Unnamed: 0" in dataframes[file].columns:
        dataframes[file].drop("Unnamed: 0", axis=1, inplace=True)

    if "match_report_url" in dataframes[file].columns:
        dataframes[file].drop("match_report_url", axis=1, inplace=True)    

    if "team1_yellow_cards" in dataframes[file].columns:
        dataframes[file].drop("team1_yellow_cards", axis=1, inplace=True)

    if "team2_yellow_cards" in dataframes[file].columns:
        dataframes[file].drop("team2_yellow_cards", axis=1, inplace=True)  
    dfss.append(dataframes[file])

    print(file)
    print(dataframes[file].shape)
    nan_counts = dataframes[file].isna().sum()
    print(nan_counts)


# Combine all the DataFrames into a single DataFrame
combined_df_2023 = pd.concat(dfss, ignore_index=True)


combined_df_2023['team1'] = combined_df_2023['team1'].str.lower()
combined_df_2023['team2'] = combined_df_2023['team2'].str.lower()
combined_df_2023.replace('', np.nan, inplace=True)
# Add a column to mark future matches
combined_df_2023['is_future_match'] = combined_df_2023['team1_goals'].isna() | combined_df_2023['team2_goals'].isna()
combined_df_2023['season'] = '2023'
# Replace empty string with NaN
combined_df_2023["team1_red_cards"].replace('', np.nan, inplace=True)
combined_df_2023["team2_red_cards"].replace('', np.nan, inplace=True)

# Replace NaN with 0
combined_df_2023["team1_red_cards"].fillna(0, inplace=True)
combined_df_2023["team2_red_cards"].fillna(0, inplace=True)

combined_df_2023['date'] = pd.to_datetime(combined_df_2023['date'], format='%Y-%m-%d', errors='coerce')
combined_df_2023.sort_values('date', inplace=True)

print(combined_df_2023.columns)
print(combined_df_2023.shape) 


dataframe2023_BRA_A.xlsx
(370, 19)
team1                      0
team2                      0
team1_goals              181
team2_goals              181
season                     0
championship               0
team1_shots_on_target    181
team1_shots_out          181
team2_shots_on_target    181
team2_shots_out          181
team1_red_cards          181
team2_red_cards          181
team1_fouls              181
team2_fouls              181
team1_corners            181
team2_corners            181
team1_total_shots        181
team2_total_shots        181
date                       0
dtype: int64
dataframe2023_SUE_A.xlsx
(230, 19)
team1                     0
team2                     0
team1_goals              80
team2_goals              80
season                    0
championship              0
team1_shots_on_target    80
team1_shots_out          80
team2_shots_on_target    80
team2_shots_out          80
team1_red_cards          80
team2_red_cards          80
team1_fouls              80
te

In [165]:
import pandas as pd
import numpy as np


combined_df_2023.sort_values('date', inplace=True)


combined_df_2023['team1_goals'] = pd.to_numeric(combined_df_2023['team1_goals'], errors='coerce')
combined_df_2023['team2_goals'] = pd.to_numeric(combined_df_2023['team2_goals'], errors='coerce')

# calculate goal differences
combined_df_2023['goal_diff_team1'] = combined_df_2023['team1_goals'] - combined_df_2023['team2_goals']
combined_df_2023['goal_diff_team2'] = combined_df_2023['team2_goals'] - combined_df_2023['team1_goals']

# calculate corners differences
combined_df_2023['corners_diff_team1'] = combined_df_2023['team1_corners'] - combined_df_2023['team2_corners']############# NEW
combined_df_2023['corners_diff_team2'] = combined_df_2023['team2_corners'] - combined_df_2023['team1_corners']############# NEW

# calculate big wins and losses
combined_df_2023['team1_big_win'] = np.where(combined_df_2023['goal_diff_team1'] >= 2, 1, 0)
combined_df_2023['team1_big_loss'] = np.where(combined_df_2023['goal_diff_team1'] <= -2, 1, 0)
combined_df_2023['team2_big_win'] = np.where(combined_df_2023['goal_diff_team2'] >= 2, 1, 0)
combined_df_2023['team2_big_loss'] = np.where(combined_df_2023['goal_diff_team2'] <= -2, 1, 0)

# calculate AH-2.5 win and losses
combined_df_2023['team1_ah-2.5_win'] = np.where(combined_df_2023['corners_diff_team1'] >= 3, 1, 0)############# NEW
combined_df_2023['team1_ah-2.5_loss'] = np.where(combined_df_2023['corners_diff_team1'] <= 2, 1, 0)############# NEW
combined_df_2023['team2_ah-2.5_win'] = np.where(combined_df_2023['corners_diff_team2'] >= 3, 1, 0)############# NEW
combined_df_2023['team2_ah-2.5_loss'] = np.where(combined_df_2023['corners_diff_team2'] <= 2, 1, 0)############# NEW


# calculate AH+2.5 win and losses
combined_df_2023['team1_ah+2.5_win'] = np.where(combined_df_2023['corners_diff_team1'] >= -2, 1, 0)############# NEW
combined_df_2023['team1_ah+2.5_loss'] = np.where(combined_df_2023['corners_diff_team1'] <= -3, 1, 0)############# NEW
combined_df_2023['team2_ah+2.5_win'] = np.where(combined_df_2023['corners_diff_team2'] >= -2, 1, 0)############# NEW
combined_df_2023['team2_ah+2.5_loss'] = np.where(combined_df_2023['corners_diff_team2'] <= -3, 1, 0)############# NEW


# calculate over4.5 win and losses
combined_df_2023['team1_over4.5'] = np.where(combined_df_2023['team1_corners'] >= 5, 1, 0)############# NEW
combined_df_2023['team1_under4.5'] = np.where(combined_df_2023['team1_corners'] <= 4, 1, 0)############# NEW
combined_df_2023['team2_over4.5'] = np.where(combined_df_2023['team2_corners'] >= 5, 1, 0)############# NEW
combined_df_2023['team2_under4.5'] = np.where(combined_df_2023['team2_corners'] <= 4, 1, 0)############# NEW

# calculate over3.5 win and losses
combined_df_2023['team1_over3.5'] = np.where(combined_df_2023['team1_corners'] >= 4, 1, 0)############# NEW
combined_df_2023['team1_under3.5'] = np.where(combined_df_2023['team1_corners'] <= 3, 1, 0)############# NEW
combined_df_2023['team2_over3.5'] = np.where(combined_df_2023['team2_corners'] >= 4, 1, 0)############# NEW
combined_df_2023['team2_under3.5'] = np.where(combined_df_2023['team2_corners'] <= 3, 1, 0)############# NEW

# calculate over6.5 win and losses
combined_df_2023['team1_over6.5'] = np.where(combined_df_2023['team1_corners'] >= 7, 1, 0)############# NEW
combined_df_2023['team1_under6.5'] = np.where(combined_df_2023['team1_corners'] <= 6, 1, 0)############# NEW
combined_df_2023['team2_over6.5'] = np.where(combined_df_2023['team2_corners'] >= 7, 1, 0)############# NEW
combined_df_2023['team2_under6.5'] = np.where(combined_df_2023['team2_corners'] <= 6, 1, 0)############# NEW


# Initialize these columns with 0
combined_df_2023['team1_big_wins_last5'] = 0
combined_df_2023['team1_big_losses_last5'] = 0
combined_df_2023['team2_big_wins_last5'] = 0
combined_df_2023['team2_big_losses_last5'] = 0

combined_df_2023['team1_ah-2.5_wins_last5'] = 0############# NEW
combined_df_2023['team1_ah-2.5_losses_last5'] = 0############# NEW
combined_df_2023['team2_ah-2.5_wins_last5'] = 0############# NEW
combined_df_2023['team2_ah-2.5_losses_last5'] = 0############# NEW

combined_df_2023['team1_ah+2.5_wins_last5'] = 0############# NEW
combined_df_2023['team1_ah+2.5_losses_last5'] = 0############# NEW
combined_df_2023['team2_ah+2.5_wins_last5'] = 0############# NEW
combined_df_2023['team2_ah+2.5_losses_last5'] = 0############# NEW

combined_df_2023['team1_over3.5_last5'] = 0############# NEW
combined_df_2023['team1_under3.5_last5'] = 0############# NEW
combined_df_2023['team2_over3.5_last5'] = 0############# NEW
combined_df_2023['team2_under3.5_last5'] = 0############# NEW

combined_df_2023['team1_over4.5_last5'] = 0############# NEW
combined_df_2023['team1_under4.5_last5'] = 0############# NEW
combined_df_2023['team2_over4.5_last5'] = 0############# NEW
combined_df_2023['team2_under4.5_last5'] = 0############# NEW

combined_df_2023['team1_over6.5_last5'] = 0############# NEW
combined_df_2023['team1_under6.5_last5'] = 0############# NEW
combined_df_2023['team2_over6.5_last5'] = 0############# NEW
combined_df_2023['team2_under6.5_last5'] = 0############# NEW


new_cols = ['avg_scr_lasts3_1_home', 'avg_scr_lasts5_1_home', 'avg_scr_lasts3_1_away',
            'avg_scr_lasts5_1_away', 'avg_conc_lasts3_1_home', 'avg_conc_lasts5_1_home',
            'avg_conc_lasts3_1_away', 'avg_conc_lasts5_1_away', 'avg_scr_lasts3_2_home',
            'avg_scr_lasts5_2_home', 'avg_scr_lasts3_2_away', 'avg_scr_lasts5_2_away',
            'avg_conc_lasts3_2_home', 'avg_conc_lasts5_2_home', 'avg_conc_lasts3_2_away',
            'avg_conc_lasts5_2_away','team1_big_wins_last5', 'team1_big_losses_last5', 
            'team2_big_wins_last5', 'team2_big_losses_last5',
            #abaixo vai ser baseado em finalizações
            'avg_total_shots_lasts5_1_home','avg_total_shots_lasts5_1_away','avg_total_shots_lasts5_2_home',
            'avg_total_shots_lasts5_2_away', 'avg_otarget_shots_lasts5_1_home','avg_otarget_shots_lasts5_1_away',
            'avg_otarget_shots_lasts5_2_home','avg_otarget_shots_lasts5_2_away','avg_out_shots_lasts5_1_home',
            'avg_out_shots_lasts5_1_away','avg_out_shots_lasts5_2_home','avg_out_shots_lasts5_2_away',
            'avg_conc_total_shots_lasts5_1_home','avg_conc_total_shots_lasts5_1_away',
            'avg_conc_total_shots_lasts5_2_home','avg_conc_total_shots_lasts5_2_away',
            #abaixo vai ser baseado em corners
            'avg_corners_lasts5_1_home','avg_corners_lasts5_1_away', 
            'avg_corners_conc_lasts5_1_home','avg_corners_conc_lasts5_1_away',
            'avg_corners_lasts5_2_home','avg_corners_lasts5_2_away', 
            'avg_corners_conc_lasts5_2_home', 'avg_corners_conc_lasts5_2_away',
            #abaixo vai ser baseado em fouls
            'avg_fouls_lasts5_1_home','avg_fouls_lasts5_1_away', 
            'avg_fouls_conc_lasts5_1_home', 'avg_fouls_conc_lasts5_1_away',
            'avg_fouls_lasts5_2_home','avg_fouls_lasts5_2_away', 
            'avg_fouls_conc_lasts5_2_home', 'avg_fouls_conc_lasts5_2_away',
            #novas colunas da v7
            'team1_ah-2.5_wins_last5', 'team1_ah-2.5_losses_last5','team2_ah-2.5_wins_last5','team2_ah-2.5_losses_last5',
            'team1_ah+2.5_wins_last5','team1_ah+2.5_losses_last5','team2_ah+2.5_wins_last5','team2_ah+2.5_losses_last5',
            'team1_over3.5_last5','team1_under3.5_last5','team2_over3.5_last5','team2_under3.5_last5',
            'team1_over4.5_last5','team1_under4.5_last5','team2_over4.5_last5','team2_under4.5_last5',
            'team1_over6.5_last5','team1_under6.5_last5','team2_over6.5_last5','team2_under6.5_last5'
            ]

# Create a dictionary with keys as column names and values as np.nan
new_cols_dict = {col: np.nan for col in new_cols}

# Add new columns to the DataFrame
combined_df_2023 = combined_df_2023.assign(**new_cols_dict)


# Iterate over each row in the DataFrame
for i, row in combined_df_2023.iterrows():
    # For each team, get their past home and away matches before the current date
    team1_matches = combined_df_2023[((combined_df_2023['team1'] == row['team1']) | (combined_df_2023['team2'] == row['team1'])) & (combined_df_2023['date'] < row['date']) & (combined_df_2023['season'] == row['season'])].sort_values(by='date')
    team2_matches = combined_df_2023[((combined_df_2023['team1'] == row['team2']) | (combined_df_2023['team2'] == row['team2'])) & (combined_df_2023['date'] < row['date']) & (combined_df_2023['season'] == row['season'])].sort_values(by='date')

    # For each team, calculate stats for last 5 matches
    if not team1_matches.empty:
        team1_matches['big_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_big_win'], team1_matches['team2_big_win'])
        team1_matches['big_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_big_loss'], team1_matches['team2_big_loss'])
        combined_df_2023.at[i, 'team1_big_wins_last5'] = team1_matches['big_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_big_losses_last5'] = team1_matches['big_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para ah-2.5 para a equipe 1
        team1_matches['ah-2.5_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah-2.5_win'], team1_matches['team2_ah-2.5_win'])
        team1_matches['ah-2.5_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah-2.5_loss'], team1_matches['team2_ah-2.5_loss'])
        combined_df_2023.at[i, 'team1_ah-2.5_wins_last5'] = team1_matches['ah-2.5_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_ah-2.5_losses_last5'] = team1_matches['ah-2.5_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para ah+2.5 para a equipe 1
        team1_matches['ah+2.5_win'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah+2.5_win'], team1_matches['team2_ah+2.5_win'])
        team1_matches['ah+2.5_loss'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_ah+2.5_loss'], team1_matches['team2_ah+2.5_loss'])
        combined_df_2023.at[i, 'team1_ah+2.5_wins_last5'] = team1_matches['ah+2.5_win'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_ah+2.5_losses_last5'] = team1_matches['ah+2.5_loss'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over3.5 para a equipe 1
        team1_matches['over3.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over3.5'], team1_matches['team2_over3.5'])
        team1_matches['under3.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under3.5'], team1_matches['team2_under3.5'])
        combined_df_2023.at[i, 'team1_over3.5_last5'] = team1_matches['over3.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under3.5_last5'] = team1_matches['under3.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over4.5 para a equipe 1
        team1_matches['over4.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over4.5'], team1_matches['team2_over4.5'])
        team1_matches['under4.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under4.5'], team1_matches['team2_under4.5'])
        combined_df_2023.at[i, 'team1_over4.5_last5'] = team1_matches['over4.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under4.5_last5'] = team1_matches['under4.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan

        # Para over6.5 para a equipe 1
        team1_matches['over6.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_over6.5'], team1_matches['team2_over6.5'])
        team1_matches['under6.5'] = np.where(team1_matches['team1'] == row['team1'], team1_matches['team1_under6.5'], team1_matches['team2_under6.5'])
        combined_df_2023.at[i, 'team1_over6.5_last5'] = team1_matches['over6.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team1_under6.5_last5'] = team1_matches['under6.5'].rolling(5).sum().iloc[-1] if len(team1_matches) >= 2 else np.nan



    if not team2_matches.empty:
        team2_matches['big_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_big_win'], team2_matches['team2_big_win'])
        team2_matches['big_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_big_loss'], team2_matches['team2_big_loss'])
        combined_df_2023.at[i, 'team2_big_wins_last5'] = team2_matches['big_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_big_losses_last5'] = team2_matches['big_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para ah-2.5 para a equipe 2
        team2_matches['ah-2.5_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah-2.5_win'], team2_matches['team2_ah-2.5_win'])
        team2_matches['ah-2.5_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah-2.5_loss'], team2_matches['team2_ah-2.5_loss'])
        combined_df_2023.at[i, 'team2_ah-2.5_wins_last5'] = team2_matches['ah-2.5_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_ah-2.5_losses_last5'] = team2_matches['ah-2.5_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para ah+2.5 para a equipe 2
        team2_matches['ah+2.5_win'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah+2.5_win'], team2_matches['team2_ah+2.5_win'])
        team2_matches['ah+2.5_loss'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_ah+2.5_loss'], team2_matches['team2_ah+2.5_loss'])
        combined_df_2023.at[i, 'team2_ah+2.5_wins_last5'] = team2_matches['ah+2.5_win'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_ah+2.5_losses_last5'] = team2_matches['ah+2.5_loss'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over3.5  para a equipe 2
        team2_matches['over3.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over3.5'], team2_matches['team2_over3.5'])
        team2_matches['under3.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under3.5'], team2_matches['team2_under3.5'])
        combined_df_2023.at[i, 'team2_over3.5_last5'] = team2_matches['over3.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under3.5_last5'] = team2_matches['under3.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over4.5  para a equipe 2
        team2_matches['over4.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over4.5'], team2_matches['team2_over4.5'])
        team2_matches['under4.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under4.5'], team2_matches['team2_under4.5'])
        combined_df_2023.at[i, 'team2_over4.5_last5'] = team2_matches['over4.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under4.5_last5'] = team2_matches['under4.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan

        # Para over6.5  para a equipe 2
        team2_matches['over6.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_over6.5'], team2_matches['team2_over6.5'])
        team2_matches['under6.5'] = np.where(team2_matches['team1'] == row['team2'], team2_matches['team1_under6.5'], team2_matches['team2_under6.5'])
        combined_df_2023.at[i, 'team2_over6.5_last5'] = team2_matches['over6.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan
        combined_df_2023.at[i, 'team2_under6.5_last5'] = team2_matches['under6.5'].rolling(5).sum().iloc[-1] if len(team2_matches) >= 2 else np.nan



    
for i, row in combined_df_2023.iterrows():
    team1_home = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team1'] == row['team1']) & (combined_df_2023['season'] == row['season'])]
    team1_away = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team2'] == row['team1']) & (combined_df_2023['season'] == row['season'])]
    
    team2_home = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team1'] == row['team2']) & (combined_df_2023['season'] == row['season'])]
    team2_away = combined_df_2023[(combined_df_2023['date'] < row['date']) & (combined_df_2023['team2'] == row['team2']) & (combined_df_2023['season'] == row['season'])]

    if not team1_home.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_1_home'] = team1_home['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_1_home'] = team1_home['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_1_home'] = team1_home['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_home['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_1_home'] = team1_home['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_1_home'] = team1_home['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_1_home'] = team1_home['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_1_home'] = team1_home['team1_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_1_home'] = team1_home['team1_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_shots_out'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_1_home'] = team1_home['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_1_home'] = team1_home['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_1_home'] = team1_home['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team1_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_1_home'] = team1_home['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_home['team2_fouls'].isna().any() else np.nan

    if not team1_away.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_1_away'] = team1_away['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_1_away'] = team1_away['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_1_away'] = team1_away['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team1_away['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_1_away'] = team1_away['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_1_away'] = team1_away['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_1_away'] = team1_away['team2_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_1_away'] = team1_away['team2_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_1_away'] = team1_away['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_1_away'] = team1_away['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_1_away'] = team1_away['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_1_away'] = team1_away['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team2_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_1_away'] = team1_away['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team1_away['team1_fouls'].isna().any() else np.nan

    if not team2_home.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_2_home'] = team2_home['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_2_home'] = team2_home['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_2_home'] = team2_home['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_home['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_2_home'] = team2_home['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_2_home'] = team2_home['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_2_home'] = team2_home['team1_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_2_home'] = team2_home['team1_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_2_home'] = team2_home['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_2_home'] = team2_home['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_2_home'] = team2_home['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_2_home'] = team2_home['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team1_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_2_home'] = team2_home['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_home['team2_fouls'].isna().any() else np.nan


    if not team2_away.empty:
        combined_df_2023.at[i, 'avg_scr_lasts3_2_away'] = team2_away['team2_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_scr_lasts5_2_away'] = team2_away['team2_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts3_2_away'] = team2_away['team1_goals'].rolling(3, min_periods=2).mean().iloc[-1] if not team2_away['team1_goals'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_lasts5_2_away'] = team2_away['team1_goals'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_goals'].isna().any() else np.nan
        ####################################### ABAIXO É SOBRE FINALIZAÇÕES ###########################################
        combined_df_2023.at[i, 'avg_total_shots_lasts5_2_away'] = team2_away['team2_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_total_shots'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_otarget_shots_lasts5_2_away'] = team2_away['team2_shots_on_target'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_shots_on_target'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_out_shots_lasts5_2_away'] = team2_away['team2_shots_out'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_shots_out'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_conc_total_shots_lasts5_2_away'] = team2_away['team1_total_shots'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_total_shots'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE CORNERS ##########################################
        combined_df_2023.at[i, 'avg_corners_lasts5_2_away'] = team2_away['team2_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_corners'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_corners_conc_lasts5_2_away'] = team2_away['team1_corners'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_corners'].isna().any() else np.nan
        ########################################## ABAIXO É SOBRE FOULS ##########################################
        combined_df_2023.at[i, 'avg_fouls_lasts5_2_away'] = team2_away['team2_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team2_fouls'].isna().any() else np.nan
        combined_df_2023.at[i, 'avg_fouls_conc_lasts5_2_away'] = team2_away['team1_fouls'].rolling(5, min_periods=2).mean().iloc[-1] if not team2_away['team1_fouls'].isna().any() else np.nan
        




combined_df_2023.shape        

(600, 120)

In [166]:
def get_result(row):
    if row['team1_goals'] > row['team2_goals']:
        return pd.Series([3, 0])
    elif row['team1_goals'] < row['team2_goals']:
        return pd.Series([0, 3])
    else:
        return pd.Series([1, 1])

combined_df_2023[['result_team1', 'result_team2']] = combined_df_2023.apply(get_result, axis=1)

def get_streak(df, result_col, results):
    result_series = df[result_col].apply(lambda x: 1 if x in results else 0)
    result_series = result_series * (result_series.groupby((result_series != result_series.shift()).cumsum()).cumcount() + 1)
    return result_series

# Create a dictionary to hold individual team dataframes
team_df_dict = {}

def get_individual_team_df(df, team_name): #teoricamente aqui deveria ser corners, mas o erro se apresentou menor assim:
    if team_name in team_df_dict:
        return team_df_dict[team_name]
        
    team_games = df[(df['team1'] == team_name) | (df['team2'] == team_name)].copy()
    team_games['team_is_team1'] = team_games['team1'] == team_name
    team_games['team_result'] = np.where(team_games['team_is_team1'], team_games['result_team1'], team_games['result_team2'])
    team_games['team_goals'] = np.where(team_games['team_is_team1'], team_games['team1_goals'], team_games['team2_goals'])
    team_games['team_redcards'] = np.where(team_games['team_is_team1'], team_games['team1_red_cards'], team_games['team2_red_cards'])

    team_games.sort_values('date', inplace=True)
    team_games['days_since_last_game'] = team_games['date'].diff().dt.days

    team_df_dict[team_name] = team_games
    return team_games

def get_team_stats(row, df):
    team1_games = get_individual_team_df(df, row['team1'])
    team2_games = get_individual_team_df(df, row['team2'])

    # Filter to include only games that occurred before the current game
    team1_games = team1_games[team1_games['date'] < row['date']]
    team2_games = team2_games[team2_games['date'] < row['date']]

    stats = {}

    if not team1_games.empty:
        stats['team1_winning_streak'] = get_streak(team1_games, 'team_result', [3]).iloc[-1]
        stats['team1_undefeated_streak'] = get_streak(team1_games, 'team_result', [1, 3]).iloc[-1]
        stats['team1_losing_streak'] = get_streak(team1_games, 'team_result', [0]).iloc[-1]
        stats['team1_without_winning_streak'] = get_streak(team1_games, 'team_result', [0, 1]).iloc[-1]
        stats['avg_points_lasts5_1'] = team1_games.tail(5)['team_result'].mean()
        stats['team1_strength'] = team1_games['team_goals'].sum() / (team1_games['team1_goals'].sum() + team1_games['team2_goals'].sum() + 0.01)
        stats['championship_points_1'] = team1_games['team_result'].sum() / len(team1_games)
        rested_4_or_more_days_1 = team1_games.tail(1)['days_since_last_game'].values[0] >= 4
        stats['rested_4_days_or_more_1'] = 1 if rested_4_or_more_days_1 else -1

    if not team2_games.empty:
        stats['team2_winning_streak'] = get_streak(team2_games, 'team_result', [3]).iloc[-1]
        stats['team2_undefeated_streak'] = get_streak(team2_games, 'team_result', [1, 3]).iloc[-1]
        stats['team2_losing_streak'] = get_streak(team2_games, 'team_result', [0]).iloc[-1]
        stats['team2_without_winning_streak'] = get_streak(team2_games, 'team_result', [0, 1]).iloc[-1]
        stats['avg_points_lasts5_2'] = team2_games.tail(5)['team_result'].mean()
        stats['team2_strength'] = team2_games['team_goals'].sum() / (team2_games['team1_goals'].sum() + team2_games['team2_goals'].sum() + 0.01)
        stats['championship_points_2'] = team2_games['team_result'].sum() / len(team2_games)
        rested_4_or_more_days_2 = team2_games.tail(1)['days_since_last_game'].values[0] >= 4
        stats['rested_4_days_or_more_2'] = 1 if rested_4_or_more_days_2 else -1

    return pd.Series(stats)

combined_df_2023 = pd.concat([combined_df_2023, combined_df_2023.apply(lambda row: get_team_stats(row, combined_df_2023), axis=1)], axis=1)

# Now, calculate the number of suspended players for the next match for each team.
for team_name in team_df_dict.keys():
    team_df = team_df_dict[team_name].copy()
    team_df['next_match_suspended_players'] = team_df['team_redcards'].shift()

    # Assign the suspended players back to the combined_df_2023.
    team1_mask = combined_df_2023['team1'] == team_name
    team2_mask = combined_df_2023['team2'] == team_name
    combined_df_2023.loc[team1_mask, 'team1_suspended_players'] = team_df.loc[team1_mask, 'next_match_suspended_players']
    combined_df_2023.loc[team2_mask, 'team2_suspended_players'] = team_df.loc[team2_mask, 'next_match_suspended_players']

combined_df_2023.shape    


(600, 140)

In [167]:
list(combined_df_2023.columns)

['team1',
 'team2',
 'team1_goals',
 'team2_goals',
 'season',
 'championship',
 'team1_shots_on_target',
 'team1_shots_out',
 'team2_shots_on_target',
 'team2_shots_out',
 'team1_red_cards',
 'team2_red_cards',
 'team1_fouls',
 'team2_fouls',
 'team1_corners',
 'team2_corners',
 'team1_total_shots',
 'team2_total_shots',
 'date',
 'is_future_match',
 'goal_diff_team1',
 'goal_diff_team2',
 'corners_diff_team1',
 'corners_diff_team2',
 'team1_big_win',
 'team1_big_loss',
 'team2_big_win',
 'team2_big_loss',
 'team1_ah-2.5_win',
 'team1_ah-2.5_loss',
 'team2_ah-2.5_win',
 'team2_ah-2.5_loss',
 'team1_ah+2.5_win',
 'team1_ah+2.5_loss',
 'team2_ah+2.5_win',
 'team2_ah+2.5_loss',
 'team1_over4.5',
 'team1_under4.5',
 'team2_over4.5',
 'team2_under4.5',
 'team1_over3.5',
 'team1_under3.5',
 'team2_over3.5',
 'team2_under3.5',
 'team1_over6.5',
 'team1_under6.5',
 'team2_over6.5',
 'team2_under6.5',
 'team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_loss

In [168]:
columns_to_drop = [
 'team1_goals',
 'team2_goals',
 'team1_shots_on_target',
 'team1_shots_out',
 'team2_shots_on_target',
 'team2_shots_out',
 'team1_red_cards',
 'team2_red_cards',
 'team1_fouls',
 'team2_fouls',
 'team1_corners',
 'team2_corners',
 'team1_total_shots',
 'team2_total_shots',
 'date',
 'is_future_match',
 'goal_diff_team1',
 'goal_diff_team2',
 'corners_diff_team1',
 'corners_diff_team2',
 'team1_big_win',
 'team1_big_loss',
 'team2_big_win',
 'team2_big_loss',
 'team1_ah-2.5_win',
 'team1_ah-2.5_loss',
 'team2_ah-2.5_win',
 'team2_ah-2.5_loss',
 'team1_ah+2.5_win',
 'team1_ah+2.5_loss',
 'team2_ah+2.5_win',
 'team2_ah+2.5_loss',
 'team1_over4.5',
 'team1_under4.5',
 'team2_over4.5',
 'team2_under4.5',
 'team1_over3.5',
 'team1_under3.5',
 'team2_over3.5',
 'team2_under3.5',
 'team1_over6.5',
 'team1_under6.5',
 'team2_over6.5',
 'team2_under6.5',
 'result_team1',
 'result_team2']

Criando future Match baseado na data de hoje

In [169]:
dataset2 = combined_df_2023.copy()

# filter the DataFrame
future_matches = dataset2[dataset2['is_future_match'] == True]
future_matches.sort_values(by='date')
print(future_matches.shape)

from datetime import datetime, timedelta

# Get yesterday's date
yesterday = datetime.now() - timedelta(days=1)

# Filter the DataFrame to include only rows with dates greater than yesterday
#future_matches = future_matches[future_matches['date'] > yesterday]

# Liste todas as colunas que você deseja verificar
columns_to_check= [col for col in future_matches.columns if col not in columns_to_drop]

# Drop as linhas com 'np.nan' nas colunas especificadas
future_matches = future_matches.dropna(subset=columns_to_check)
future_matches_calculado = future_matches.drop(columns_to_drop,axis=1)
future_matches_calculado = future_matches_calculado.drop('season',axis=1)
"""# Contando os valores NaN em cada coluna
nan_counts = future_matches.isna().sum()

# Transformando em uma lista de pares (nome da coluna, contagem de np.nan)
nan_list = list(nan_counts.items())

# Percorrendo a lista e imprimindo cada valor individualmente com o nome da coluna
for col_name, nan_count in nan_list:
    print(f'{col_name}: {nan_count}')"""
    
print(future_matches_calculado.shape)

(261, 140)
(18, 93)


In [171]:
#CORTAR AS LINHAS COM MATCHES FUTUROS AQUI

# Replace empty strings with NaN
print(dataset2.shape)
dataset2.replace('', np.nan, inplace=True)
print(f"A quantidade de np.nan em linhas eram {dataset2.isna().sum().sum()}")
# Remove rows that contain any missing values
dataset2.dropna(inplace=True)

import numpy as np

counter = 0  # Initialize counter
for index, row in dataset2.iterrows():
    if row.isnull().any() or row.eq('').any():
        print(f"Row {index} contains 'NaN' or an empty value.")
        counter += 1  # Increase counter if condition is met

print(f"Total number of rows with 'NaN' or an empty value: {counter}")
dataset2.shape


(246, 140)
A quantidade de np.nan em linhas eram 0
Total number of rows with 'NaN' or an empty value: 0


(246, 140)

Concatenando os 2

In [172]:
datatotal = pd.concat([dataset, dataset2], ignore_index=True)#mudei o 'dataset' por combined_df_13c_new

datatotal.sort_values(by='date', inplace=True)

if 'team1_yellow_cards' in datatotal.columns:
    datatotal = datatotal.drop(['team1_yellow_cards'], axis=1)

if 'team2_yellow_cards' in datatotal.columns:
    datatotal = datatotal.drop(['team2_yellow_cards'], axis=1)

# Substituir valores maiores que 15 por 15 na coluna 'team1_corners'
datatotal.loc[datatotal['team1_corners'] > 15, 'team1_corners'] = 15

# Substituir valores maiores que 15 por 15 na coluna 'team2_corners'
datatotal.loc[datatotal['team2_corners'] > 15, 'team2_corners'] = 15    

datatotal

,championship,date,team1,team2,team1_goals,team2_goals,team1_total_shots,team2_total_shots,team1_shots_on_target,team2_shots_on_target,...,team1_winning_streak,team1_without_winning_streak,team2_losing_streak,team2_strength,team2_undefeated_streak,team2_winning_streak,team2_without_winning_streak,team1_suspended_players,team2_suspended_players,is_future_match
0,E0,2002-09-14,Charlton,Arsenal,0.0,3.0,9.0,10.0,3.0,8.0,...,0.0,1.0,0.0,0.666297,5.0,1.0,0.0,0.0,0.0,NaN
1,E0,2002-09-14,Everton,Middlesbrough,2.0,1.0,13.0,10.0,8.0,5.0,...,0.0,3.0,0.0,0.665927,1.0,1.0,0.0,0.0,0.0,NaN
2,E0,2002-09-14,Leeds,Man United,1.0,0.0,8.0,6.0,2.0,5.0,...,1.0,0.0,1.0,0.554939,0.0,0.0,1.0,0.0,0.0,NaN
3,E0,2002-09-14,West Brom,Southampton,1.0,0.0,11.0,10.0,7.0,5.0,...,2.0,0.0,0.0,0.332963,1.0,1.0,0.0,0.0,0.0,NaN
4,E0,2002-09-15,Man City,Blackburn,2.0,2.0,15.0,12.0,7.0,8.0,...,0.0,1.0,2.0,0.454133,0.0,0.0,3.0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67081,BRA A,2023-08-20,bahia,red bull bragantino,4.0,0.0,9.0,10.0,6.0,8.0,...,0.0,7.0,0.0,0.613497,4.0,0.0,1.0,0.0,0.0,False
67082,BRA A,2023-08-20,santos,grêmio,2.0,1.0,15.0,6.0,8.0,4.0,...,0.0,5.0,0.0,0.531802,1.0,1.0,0.0,0.0,0.0,False
67083,SUE A,2023-08-20,kalmar,hammarby if,0.0,0.0,2.0,4.0,1.0,2.0,...,0.0,3.0,0.0,0.469292,1.0,1.0,0.0,0.0,0.0,False
67084,SUE A,2023-08-21,norrkoping,aik,3.0,1.0,10.0,12.0,5.0,8.0,...,4.0,0.0,0.0,0.438917,5.0,0.0,2.0,0.0,0.0,False


In [173]:
champ_uniques = datatotal['championship'].unique()
print(champ_uniques)

['E0' 'SC0' 'E3' 'E2' 'E1' 'I1' 'SP1' 'D1' 'F1' 'D2' 'P1' 'SP2' 'T1' 'I2'
 'N1' 'F2' 'B1' 'G1' 'SUE A' 'BRA A']


Converta Categorias em IDs Numéricos

In [174]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Suponho que 'datatotal' e 'future_matches' já estejam definidos e tenham as mesmas colunas relevantes

# Treinar o LabelEncoder com 'datatotal' e 'future_matches'
le_teams = LabelEncoder().fit(pd.concat([datatotal['team1'], datatotal['team2'], future_matches_calculado['team1'], future_matches_calculado['team2']]).astype(str))
le_champ = LabelEncoder().fit(pd.concat([datatotal['championship'], future_matches_calculado['championship']]).astype(str))

# Aplicar o LabelEncoder a 'datatotal'
datatotal['team1'] = le_teams.transform(datatotal['team1'].astype(str))
datatotal['team2'] = le_teams.transform(datatotal['team2'].astype(str))
datatotal['championship'] = le_champ.transform(datatotal['championship'].astype(str))

# Agora, aplicar o mesmo LabelEncoder a 'future_matches'
future_matches_calculado['team1'] = le_teams.transform(future_matches_calculado['team1'].astype(str))
future_matches_calculado['team2'] = le_teams.transform(future_matches_calculado['team2'].astype(str))
future_matches_calculado['championship'] = le_champ.transform(future_matches_calculado['championship'].astype(str))



In [175]:
future_matches_calculado.head()

,team1,team2,championship,team1_big_wins_last5,team1_big_losses_last5,team2_big_wins_last5,team2_big_losses_last5,team1_ah-2.5_wins_last5,team1_ah-2.5_losses_last5,team2_ah-2.5_wins_last5,...,team1_undefeated_streak,team1_winning_streak,team1_without_winning_streak,team2_losing_streak,team2_strength,team2_undefeated_streak,team2_winning_streak,team2_without_winning_streak,team1_suspended_players,team2_suspended_players
190,475,485,1,2.0,0.0,0.0,0.0,1.0,4.0,1.0,...,6.0,0.0,1.0,0.0,0.386276,6.0,0.0,1.0,0.0,0.0
189,482,492,1,0.0,1.0,0.0,1.0,2.0,3.0,1.0,...,2.0,1.0,0.0,2.0,0.410151,0.0,0.0,8.0,0.0,0.0
197,486,477,1,0.0,0.0,0.0,0.0,0.0,5.0,3.0,...,0.0,0.0,1.0,0.0,0.483715,1.0,0.0,6.0,1.0,0.0
192,498,478,1,1.0,1.0,1.0,2.0,3.0,2.0,2.0,...,0.0,0.0,2.0,2.0,0.476077,0.0,0.0,2.0,0.0,0.0
524,495,488,18,1.0,1.0,1.0,0.0,1.0,4.0,3.0,...,0.0,0.0,1.0,0.0,0.469292,2.0,0.0,1.0,0.0,0.0


In [176]:
n_teams = len(le_teams.classes_)
n_champ = len(le_champ.classes_)
print(n_teams)
print(n_champ)

503
20


In [177]:
datatotal['season'] = datatotal['season'].astype('float64')


Separando X e Y

In [178]:
X = datatotal.drop(columns_to_drop, axis=1)
y1 = datatotal['team1_corners']
y2 = datatotal['team2_corners']
X.shape

(67086, 94)

In [179]:
list(X.dtypes)

[dtype('int32'),
 dtype('int32'),
 dtype('int32'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),

In [180]:
list(X.columns)

['championship',
 'team1',
 'team2',
 'season',
 'team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_losses_last5',
 'team1_ah-2.5_wins_last5',
 'team1_ah-2.5_losses_last5',
 'team2_ah-2.5_wins_last5',
 'team2_ah-2.5_losses_last5',
 'team1_ah+2.5_wins_last5',
 'team1_ah+2.5_losses_last5',
 'team2_ah+2.5_wins_last5',
 'team2_ah+2.5_losses_last5',
 'team1_over3.5_last5',
 'team1_under3.5_last5',
 'team2_over3.5_last5',
 'team2_under3.5_last5',
 'team1_over4.5_last5',
 'team1_under4.5_last5',
 'team2_over4.5_last5',
 'team2_under4.5_last5',
 'team1_over6.5_last5',
 'team1_under6.5_last5',
 'team2_over6.5_last5',
 'team2_under6.5_last5',
 'avg_scr_lasts3_1_home',
 'avg_scr_lasts5_1_home',
 'avg_scr_lasts3_1_away',
 'avg_scr_lasts5_1_away',
 'avg_conc_lasts3_1_home',
 'avg_conc_lasts5_1_home',
 'avg_conc_lasts3_1_away',
 'avg_conc_lasts5_1_away',
 'avg_scr_lasts3_2_home',
 'avg_scr_lasts5_2_home',
 'avg_scr_lasts3_2_away',
 'avg_scr_lasts5_2_away',
 'avg_

In [181]:
y1_uniques = datatotal['team1_corners'].unique()
print(y1_uniques)

[ 1. 10.  4.  0. 15.  9.  8.  5.  6.  7.  3. 12. 11. 14.  2. 13.]


In [182]:
# Contar ocorrências para 'y1'
y1_counts = y1.value_counts().sort_index().reindex(range(0, 31), fill_value=0)
print("Contagem de ocorrências para y1:")
print(y1_counts)

# Contar ocorrências para 'y2'
y2_counts = y2.value_counts().sort_index().reindex(range(0, 31), fill_value=0)
print("\nContagem de ocorrências para y2:")
print(y2_counts)


Contagem de ocorrências para y1:
team1_corners
0      695
1     2435
2     5020
3     7572
4     9206
5     9501
6     8621
7     7231
8     5569
9     3947
10    2772
11    1817
12    1112
13     710
14     416
15     462
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
Name: count, dtype: int64

Contagem de ocorrências para y2:
team2_corners
0      1556
1      4938
2      8146
3     10270
4     10632
5      9468
6      7381
7      5625
8      3580
9      2335
10     1369
11      847
12      415
13      283
14      137
15      104
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
30        0
Name: count, dtype: int64


Separando o treino e o teste e a normalização

In [183]:
from sklearn.preprocessing import StandardScaler

# Dividindo os dados com base na coluna 'season'
X_train1 = X[X['season'] < 2022].drop(['team1', 'team2', 'championship', 'season'], axis=1)
X_test1 = X[X['season'] >= 2022].drop(['team1', 'team2', 'championship', 'season'], axis=1)
y_train1 = y1[X['season'] < 2022]
y_test1 = y1[X['season'] >= 2022]

X_train2 = X[X['season'] < 2022].drop(['team1', 'team2', 'championship', 'season'], axis=1)
X_test2 = X[X['season'] >= 2022].drop(['team1', 'team2', 'championship', 'season'], axis=1)
y_train2 = y2[X['season'] < 2022]
y_test2 = y2[X['season'] >= 2022]

# Escalando os dados
scaler = StandardScaler()

# Ajustar o escalonador com base no conjunto de treinamento
scaler.fit(X_train1)


# Escalando apenas as colunas que você quer (ajuste isso conforme suas necessidades)
cols_to_scale = [col for col in future_matches_calculado.columns if col not in ['team1', 'team2', 'championship']]
scaler = StandardScaler().fit(X_train1[cols_to_scale])

# Aplicar o escalonamento
future_matches_calculado_scaled = future_matches_calculado.copy()
future_matches_calculado_scaled[cols_to_scale] = scaler.transform(future_matches_calculado[cols_to_scale])

# Agora, você pode usar future_matches_calculado_scaled para fazer previsões




# Transformar os conjuntos de treinamento e teste
X_train1_scaled = scaler.transform(X_train1)
X_test1_scaled = scaler.transform(X_test1)

X_train2_scaled = scaler.transform(X_train2)
X_test2_scaled = scaler.transform(X_test2)

# Preparar as colunas para o embedding
X_train1_embed = X[X['season'] < 2022][['team1', 'team2', 'championship']]
X_test1_embed = X[X['season'] >= 2022][['team1', 'team2', 'championship']]

X_train2_embed = X[X['season'] < 2022][['team1', 'team2', 'championship']]
X_test2_embed = X[X['season'] >= 2022][['team1', 'team2', 'championship']]


In [184]:
list(X_train1.columns)

['team1_big_wins_last5',
 'team1_big_losses_last5',
 'team2_big_wins_last5',
 'team2_big_losses_last5',
 'team1_ah-2.5_wins_last5',
 'team1_ah-2.5_losses_last5',
 'team2_ah-2.5_wins_last5',
 'team2_ah-2.5_losses_last5',
 'team1_ah+2.5_wins_last5',
 'team1_ah+2.5_losses_last5',
 'team2_ah+2.5_wins_last5',
 'team2_ah+2.5_losses_last5',
 'team1_over3.5_last5',
 'team1_under3.5_last5',
 'team2_over3.5_last5',
 'team2_under3.5_last5',
 'team1_over4.5_last5',
 'team1_under4.5_last5',
 'team2_over4.5_last5',
 'team2_under4.5_last5',
 'team1_over6.5_last5',
 'team1_under6.5_last5',
 'team2_over6.5_last5',
 'team2_under6.5_last5',
 'avg_scr_lasts3_1_home',
 'avg_scr_lasts5_1_home',
 'avg_scr_lasts3_1_away',
 'avg_scr_lasts5_1_away',
 'avg_conc_lasts3_1_home',
 'avg_conc_lasts5_1_home',
 'avg_conc_lasts3_1_away',
 'avg_conc_lasts5_1_away',
 'avg_scr_lasts3_2_home',
 'avg_scr_lasts5_2_home',
 'avg_scr_lasts3_2_away',
 'avg_scr_lasts5_2_away',
 'avg_conc_lasts3_2_home',
 'avg_conc_lasts5_2_home',


In [185]:
print(X_train1_scaled.shape)
print(X_test1_scaled.shape)
print(y_train1.shape)
print(y_test1.shape)
print(X_train2_scaled.shape)
print(X_test2_scaled.shape)
print(y_train2.shape)
print(y_test2.shape)
print(X_train1_embed.shape)
print(X_test1_embed.shape)
print(X_train2_embed.shape)
print(X_test2_embed.shape)
print(future_matches_calculado_scaled.shape)

(61353, 90)
(5733, 90)
(61353,)
(5733,)
(61353, 90)
(5733, 90)
(61353,)
(5733,)
(61353, 3)
(5733, 3)
(61353, 3)
(5733, 3)
(18, 93)


Função de perda

In [235]:
import tensorflow as tf

def custom_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])

    avg = tf.reduce_mean(y_true)
    abs_error = tf.abs(y_true - y_pred)
    distance_to_avg = tf.abs(y_pred - avg)
    reward = tf.math.log(distance_to_avg + 3)
    penalty = 2 * abs_error / (distance_to_avg + 3)
    
    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff = tf.math.tanh(penalty - reward)
    
    # Somando com abs_error
    custom_loss_value = abs_error + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    custom_loss_value = tf.maximum(custom_loss_value, 0.0001)
    
    return tf.reduce_mean(custom_loss_value)

Testes de funções de erro

In [251]:
import tensorflow as tf
import numpy as np

def custom_loss_test(y_true, y_pred):


    avg = 4.6
    abs_error = tf.abs(y_true - y_pred)
    distance_to_avg = tf.abs(y_pred - avg)
    reward = tf.math.log(distance_to_avg + 3)
    penalty = 2 * abs_error / (distance_to_avg + 3)
    
    # Aplicando a função tanh ao resultado de (penalty - reward)
    normalized_diff = tf.math.tanh(penalty - reward)
    
    # Somando com abs_error
    custom_loss_value = abs_error + normalized_diff
    
    # Garantindo que o valor mínimo da perda seja 0.0001
    custom_loss_value = tf.maximum(custom_loss_value, 0.0001)
    
    return tf.reduce_mean(custom_loss_value)

# Simulando 20 cenários diferentes
for i in range(20):
    y_true = tf.constant(np.random.randint(0, 16), dtype=tf.float32)
    y_pred = tf.constant(np.random.uniform(0, 16), dtype=tf.float32)
    
    loss_value = custom_loss_test(y_true, y_pred)
    
    print(f"Scenario {i+1}:")
    print(f"y_true: {y_true.numpy()}")
    print(f"y_pred: {y_pred.numpy()}")
    print(f"Custom Loss Value: {loss_value.numpy():.4f}")

    print("="*25)


Scenario 1:
y_true: 13.0
y_pred: 1.1799863576889038
Custom Loss Value: 12.7691
Scenario 2:
y_true: 5.0
y_pred: 11.976266860961914
Custom Loss Value: 6.2168
Scenario 3:
y_true: 15.0
y_pred: 5.410640716552734
Custom Loss Value: 10.5881
Scenario 4:
y_true: 8.0
y_pred: 12.097575187683105
Custom Loss Value: 3.1805
Scenario 5:
y_true: 9.0
y_pred: 0.09384145587682724
Custom Loss Value: 9.2490
Scenario 6:
y_true: 8.0
y_pred: 13.799418449401855
Custom Loss Value: 4.8855
Scenario 7:
y_true: 11.0
y_pred: 6.310484409332275
Custom Loss Value: 5.1042
Scenario 8:
y_true: 13.0
y_pred: 4.798759460449219
Custom Loss Value: 9.2005
Scenario 9:
y_true: 5.0
y_pred: 9.279520034790039
Custom Loss Value: 3.5517
Scenario 10:
y_true: 10.0
y_pred: 7.56310510635376
Custom Loss Value: 1.6890
Scenario 11:
y_true: 6.0
y_pred: 11.377161026000977
Custom Loss Value: 4.5497
Scenario 12:
y_true: 10.0
y_pred: 1.765364170074463
Custom Loss Value: 9.0199
Scenario 13:
y_true: 13.0
y_pred: 7.460981369018555
Custom Loss Value: 

In [288]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam

# ==== Parte 1: Definindo os Inputs para o Embedding ====
# Estes são os inputs que vão alimentar os embeddings. 
# Cada input tem a dimensão de (1,) porque cada jogo tem exatamente um 'team1', um 'team2', e um 'championship'.
team1_input = Input(shape=(1,), name='Team1-Input')
team2_input = Input(shape=(1,), name='Team2-Input')
champ_input = Input(shape=(1,), name='Championship-Input')

# ==== Parte 2: Criando os Embeddings ====
# n_teams e n_champ são o número de times e campeonatos únicos, respectivamente.
# O output_dim é um hiperparâmetro para você ajustar. Ele define o tamanho do espaço de embedding.

# Embedding para o time 1
team1_embedding = Embedding(input_dim=n_teams, output_dim=50, name='Team1-Embedding')(team1_input)  # output_dim ajustável

# Embedding para o time 2
team2_embedding = Embedding(input_dim=n_teams, output_dim=50, name='Team2-Embedding')(team2_input)  # output_dim ajustável

# Embedding para o campeonato
champ_embedding = Embedding(input_dim=n_champ, output_dim=5, name='Championship-Embedding')(champ_input)  # output_dim ajustável

# ==== Parte 3: Achatando os Embeddings ====
# Cada embedding precisa ser achatado para ser concatenado posteriormente
team1_embedding = Flatten()(team1_embedding)
team2_embedding = Flatten()(team2_embedding)
champ_embedding = Flatten()(champ_embedding)

# ==== Parte 4: Outras Características ====
# Este é o input para as outras características (já escaladas) do seu conjunto de dados.
other_features_input = Input(shape=(X_train1_scaled.shape[1],), name='Other-Features-Input')

# ==== Parte 5: Concatenando Tudo ====
# Aqui, todos os embeddings e as outras características são concatenados em um único vetor
merged = Concatenate()([team1_embedding, team2_embedding, champ_embedding, other_features_input])

# ==== Parte 6: Camadas Ocultas ====
# Estes são os neurônios e camadas totalmente conectadas (Dense) onde a "aprendizagem" real acontece.
# Você pode ajustar o número de neurônios, a função de ativação, e outros hiperparâmetros aqui.
from tensorflow.keras.regularizers import l1, l2

hidden_layer = Dense(512, activation='relu', kernel_regularizer=l2(0.0001))(merged)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(256, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(128, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(64, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(32, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

hidden_layer = Dense(16, activation='tanh')(hidden_layer)
hidden_layer = Dropout(0.2)(hidden_layer)

#hidden_layer = Dense(8, activation='tanh')(hidden_layer)
#hidden_layer = Dropout(0.2)(hidden_layer)

# ==== Parte 7: Camada de Saída ====
# Esta é a camada de saída. A função de ativação 'linear' é usada para regressão.
output = Dense(1, activation='linear', name='Output-Layer')(hidden_layer)

# ==== Parte 8: Compilando o Modelo ====
# Finalmente, o modelo é compilado. O otimizador Adam é usado, com uma taxa de aprendizagem de 0.001.
# A perda é definida como 'mean_squared_error', que é comum para problemas de regressão.
model = Model(
    inputs=[team1_input, team2_input, champ_input, other_features_input], 
    outputs=[output]
)


In [289]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Callbacks
#model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_mean_absolute_error', patience=10, verbose=1, mode='min')
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, mode='min')

callbacks = [early_stopping]

# 
from tensorflow.keras.models import clone_model

model1 = clone_model(model)
model1.compile(optimizer=Adam(0.001), loss=custom_loss, metrics=['mean_absolute_error'])

model2 = clone_model(model)
model2.compile(optimizer=Adam(0.001), loss=custom_loss, metrics=['mean_absolute_error'])

# Treinamento do modelo (precisará ajustar os conjuntos de entrada e saída)
history1 = model1.fit(
    [X_train1_embed['team1'], X_train1_embed['team2'], X_train1_embed['championship'], X_train1_scaled], 
    y_train1, 
    epochs=100,  # ajuste o número de épocas conforme necessário
    batch_size=128,  # ajuste o tamanho do lote conforme necessário
    verbose=1, 
    validation_split=0.2,  # usando 20% dos dados para validação; ajuste conforme necessário
    callbacks=callbacks
)

history2 = model2.fit(
    [X_train2_embed['team1'], X_train2_embed['team2'], X_train2_embed['championship'], X_train2_scaled], 
    y_train2, 
    epochs=100,  # ajuste o número de épocas conforme necessário
    batch_size=128,  # ajuste o tamanho do lote conforme necessário
    verbose=1, 
    validation_split=0.2,  # usando 20% dos dados para validação; ajuste conforme necessário
    callbacks=callbacks
)


Epoch 1/100
384/384 [==============================] - 6s 11ms/step - loss: 2.3696 - mean_absolute_error: 2.4618 - val_loss: 2.1255 - val_mean_absolute_error: 2.1901
Epoch 2/100
384/384 [==============================] - 4s 10ms/step - loss: 2.2732 - mean_absolute_error: 2.3855 - val_loss: 2.1107 - val_mean_absolute_error: 2.2501
Epoch 3/100
384/384 [==============================] - 4s 10ms/step - loss: 2.2465 - mean_absolute_error: 2.3803 - val_loss: 2.0924 - val_mean_absolute_error: 2.2127
Epoch 4/100
384/384 [==============================] - 4s 10ms/step - loss: 2.2285 - mean_absolute_error: 2.3751 - val_loss: 2.0847 - val_mean_absolute_error: 2.2248
Epoch 5/100
384/384 [==============================] - 4s 10ms/step - loss: 2.2118 - mean_absolute_error: 2.3642 - val_loss: 2.0759 - val_mean_absolute_error: 2.2569
Epoch 6/100
384/384 [==============================] - 4s 10ms/step - loss: 2.1937 - mean_absolute_error: 2.3563 - val_loss: 2.1102 - val_mean_absolute_error: 2.2329
Epoc

In [290]:
best_val_mae1 = min(history1.history['val_mean_absolute_error'])
best_val_mae2 = min(history2.history['val_mean_absolute_error'])
print(best_val_mae1)
print(best_val_mae2)


2.1901447772979736
1.996545672416687


In [291]:
future_matches_calculado_scaled.shape

(18, 93)

In [292]:
future_matches_calculado_scaled.head()

,team1,team2,championship,team1_big_wins_last5,team1_big_losses_last5,team2_big_wins_last5,team2_big_losses_last5,team1_ah-2.5_wins_last5,team1_ah-2.5_losses_last5,team2_ah-2.5_wins_last5,...,team1_undefeated_streak,team1_winning_streak,team1_without_winning_streak,team2_losing_streak,team2_strength,team2_undefeated_streak,team2_winning_streak,team2_without_winning_streak,team1_suspended_players,team2_suspended_players
190,475,485,1,1.257310,-0.986648,-0.939662,-0.950273,-0.333442,0.333442,-0.405417,...,1.228464,-0.478941,-0.389724,-0.525751,-1.372864,1.143038,-0.569612,-0.314683,-0.334677,-0.305174
189,482,492,1,-0.907378,0.123596,-0.939662,0.184672,0.596325,-0.596325,-0.405417,...,0.006032,0.327638,-0.759254,1.364236,-1.075121,-0.661135,-0.569612,2.313513,-0.334677,-0.305174
197,486,477,1,-0.907378,-0.986648,-0.939662,-0.950273,-1.263209,1.263209,1.415019,...,-0.605184,-0.478941,-0.389724,-0.525751,-0.157732,-0.360440,-0.569612,1.562600,2.587813,-0.305174
192,498,478,1,0.174966,0.123596,0.117516,1.319616,1.526092,-1.526092,0.504801,...,-0.605184,-0.478941,-0.020195,1.364236,-0.252980,-0.661135,-0.569612,0.060774,-0.334677,-0.305174
524,495,488,18,0.174966,0.123596,0.117516,-0.950273,-0.333442,0.333442,1.415019,...,-0.605184,-0.478941,-0.389724,-0.525751,-0.337596,-0.059744,-0.569612,-0.314683,-0.334677,-0.305174


Output!!!!!

In [293]:
#STEP 1

# Inicializando um dicionário vazio para armazenar as previsões
corner_predictions = {}

# Iterar sobre cada linha em 'future_matches' e 'future_matches_calculado_scaled'
for (index1, row1), (index2, row2) in zip(future_matches.iterrows(), future_matches_calculado_scaled.iterrows()):
    # Prepare os dados de entrada para a previsão
    team1_input_data = np.array([[row2['team1']]], dtype=np.float32)
    team2_input_data = np.array([[row2['team2']]], dtype=np.float32)
    champ_input_data = np.array([[row2['championship']]], dtype=np.float32)
    
    # Certifique-se de que 'other_features_data' contém todas as outras características na mesma ordem que foram usadas para treinar o modelo
    other_features_data = np.array([row2.drop(['team1', 'team2', 'championship']).astype(np.float32)])
    
    # Faça a previsão usando 'model1' e 'model2'
    pred1 = model1.predict([team1_input_data, team2_input_data, champ_input_data, other_features_data])
    pred2 = model2.predict([team1_input_data, team2_input_data, champ_input_data, other_features_data])
    
    # Armazenar as previsões no dicionário
    corner_predictions[row1['team1']] = pred1[0][0]
    corner_predictions[row1['team2']] = pred2[0][0]
    
    # Imprimir as previsões
    print(f"Prediction for {row1['team1']}: {pred1[0][0]} corners")
    print(f"Prediction for {row1['team2']}: {pred2[0][0]} corners")

# Exibindo o dicionário de previsões
print("Corner Predictions:", corner_predictions)


1/1 [==============================] - 0s 120ms/step


1/1 [==============================] - 0s 93ms/step
Prediction for corinthians: 4.193365097045898 corners
Prediction for goiás: 3.533968448638916 corners
1/1 [==============================] - 0s 16ms/step
Prediction for flamengo: 7.220284938812256 corners
Prediction for internacional: 2.8095383644104004 corners
1/1 [==============================] - 0s 22ms/step
Prediction for grêmio: 4.431801795959473 corners
Prediction for cruzeiro: 5.8556952476501465 corners
1/1 [==============================] - 0s 16ms/step
Prediction for red bull bragantino: 7.220280647277832 corners
Prediction for cuiabá: 2.871609687805176 corners
1/1 [==============================] - 0s 31ms/step
Prediction for mjallby: 3.866453170776367 corners
Prediction for hammarby if: 5.855721950531006 corners
1/1 [==============================] - 0s 22ms/step
Prediction for atlético mineiro: 7.220322608947754 corners
Prediction for santos: 2.8193790912628174 corners
1/1 [==============================] - 0s 22ms/step
P

In [294]:
# Transformar o dicionário em uma lista de listas, quebrando a cada 2 itens
items = list(corner_predictions.items())
rows = [items[i:i + 2] for i in range(0, len(items), 2)]

# Criar um DataFrame a partir da lista de listas
df = pd.DataFrame(rows, columns=['Team1', 'Team2'])

# Separar as tuplas em duas colunas separadas para os nomes das equipes e as previsões
df[['Team1_name', 'Team1_prediction']] = pd.DataFrame(df['Team1'].tolist(), index=df.index)
df[['Team2_name', 'Team2_prediction']] = pd.DataFrame(df['Team2'].tolist(), index=df.index)

# Descartar as colunas originais e reordenar
df = df[['Team1_name', 'Team1_prediction', 'Team2_name', 'Team2_prediction']]


df

,Team1_name,Team1_prediction,Team2_name,Team2_prediction
0,corinthians,4.193365,goiás,3.533968
1,flamengo,7.220285,internacional,2.809538
2,grêmio,4.431802,cruzeiro,5.855695
3,red bull bragantino,7.220281,cuiabá,2.871610
4,mjallby,3.866453,hammarby if,5.855722
5,atlético mineiro,7.220323,santos,2.819379
6,ifk gotemburgo,4.043890,bk hacken,5.549619
7,djurgardens if,7.219966,degerfors if,2.837492
8,fortaleza,6.853286,coritiba,2.832126
9,ifk varnamo,6.734648,halmstad,3.319602


In [295]:
df.shape

(18, 4)

In [296]:
#STEP 2

from scipy.stats import poisson
import numpy as np
import pandas as pd

def poisson_probabilities(lam, max_corners=15):
    probs = [poisson.pmf(k, lam) for k in range(max_corners + 1)]
    probs.append(1 - sum(probs))
    return probs

def calculate_probabilities(team1_corners_prediction, team2_corners_prediction):
    team1_corners_probabilities = poisson_probabilities(team1_corners_prediction)
    team2_corners_probabilities = poisson_probabilities(team2_corners_prediction)

    joint_prob_matrix = np.outer(team1_corners_probabilities, team2_corners_probabilities)

    team1_win_prob = np.sum(np.tril(joint_prob_matrix, -1))
    draw_prob = np.sum(np.diag(joint_prob_matrix))
    team2_win_prob = np.sum(np.triu(joint_prob_matrix, 1))

    team1_minus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=3])
    team1_plus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=-2])
    team2_minus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=3])
    team2_plus25_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=-2])

    team1_minus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=2])
    team1_plus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if i-j>=-1])
    team2_minus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=2])
    team2_plus15_prob = np.sum([joint_prob_matrix[i,j] for i in range(joint_prob_matrix.shape[0]) for j in range(joint_prob_matrix.shape[1]) if j-i>=-1])

    team1_over45_prob = 1 - sum(team1_corners_probabilities[:5])
    team1_under45_prob = sum(team1_corners_probabilities[:5])
    team2_over45_prob = 1 - sum(team2_corners_probabilities[:5])
    team2_under45_prob = sum(team2_corners_probabilities[:5])

    team1_over55_prob = 1 - sum(team1_corners_probabilities[:6])
    team1_under55_prob = sum(team1_corners_probabilities[:6])
    team2_over55_prob = 1 - sum(team2_corners_probabilities[:6])
    team2_under55_prob = sum(team2_corners_probabilities[:6])

    team1_over65_prob = 1 - sum(team1_corners_probabilities[:7])
    team1_under65_prob = sum(team1_corners_probabilities[:7])
    team2_over65_prob = 1 - sum(team2_corners_probabilities[:7])
    team2_under65_prob = sum(team2_corners_probabilities[:7])
    
    return (team1_win_prob, draw_prob, team2_win_prob,
        team1_minus15_prob, team1_plus15_prob, team2_minus15_prob, team2_plus15_prob,
        team1_minus25_prob, team1_plus25_prob, team2_minus25_prob, team2_plus25_prob,
        team1_over45_prob, team1_under45_prob, team2_over45_prob, team2_under45_prob,
        team1_over55_prob, team1_under55_prob, team2_over55_prob, team2_under55_prob,
        team1_over65_prob, team1_under65_prob, team2_over65_prob, team2_under65_prob)





In [297]:
# Inicializando o DataFrame para armazenar as odds
odds_df = pd.DataFrame(columns=['Team 1', 'Team 2', 'Team 1 Win Odd', 'Draw Odd', 'Team 2 Win Odd',
                                'Team 1 -1.5 Odd', 'Team 1 +1.5 Odd', 'Team 2 -1.5 Odd', 'Team 2 +1.5 Odd',
                                'Team 1 -2.5 Odd', 'Team 1 +2.5 Odd', 'Team 2 -2.5 Odd', 'Team 2 +2.5 Odd',
                                'Team 1 Over 4.5', 'Team 1 Under 4.5', 'Team 2 Over 4.5', 'Team 2 Under 4.5',
                                'Team 1 Over 5.5', 'Team 1 Under 5.5', 'Team 2 Over 5.5', 'Team 2 Under 5.5',
                                'Team 1 Over 6.5', 'Team 1 Under 6.5', 'Team 2 Over 6.5', 'Team 2 Under 6.5'])

# Iterar sobre cada linha no DataFrame 'df'
for index, row in df.iterrows():
    team1_name = row['Team1_name']
    team1_prediction = row['Team1_prediction']
    team2_name = row['Team2_name']
    team2_prediction = row['Team2_prediction']
    
    # Calculando as probabilidades usando a função do STEP 2
    probabilities = calculate_probabilities(team1_prediction, team2_prediction)
    
    # Calculando as odds
    odds = [1 / prob for prob in probabilities]
    
    # Adicionando as odds ao DataFrame
    odds_df.loc[len(odds_df)] = [team1_name, team2_name] + odds

    # Inicializando as novas colunas
odds_df['date'] = None
odds_df['championship'] = None

# Preenchendo as novas colunas
for index, row in odds_df.iterrows():
    team1 = row['Team 1']
    team2 = row['Team 2']
    
    # Encontrando a linha correspondente em 'future_matches'
    match_row = future_matches[(future_matches['team1'] == team1) & (future_matches['team2'] == team2)]
    
    if not match_row.empty:
        # Se encontrarmos uma linha correspondente, atualizamos 'odds_df'
        odds_df.at[index, 'date'] = match_row.iloc[0]['date']
        odds_df.at[index, 'championship'] = match_row.iloc[0]['championship']

# Converte a coluna 'date' para o tipo de data do pandas
odds_df['date'] = pd.to_datetime(odds_df['date'])

# Formata a coluna 'date' para o formato de data brasileiro (dd/mm/yyyy)
odds_df['date'] = odds_df['date'].dt.strftime('%d/%m/%Y')

odds_df = odds_df.sort_values(['championship', 'date'])

# Lista das colunas atuais
cols = odds_df.columns.tolist()

# Removendo 'date' e 'championship' da lista
cols.remove('date')
cols.remove('championship')

# Reordenando as colunas
new_cols = ['date', 'championship'] + cols

# Atualizando o DataFrame com a nova ordem de colunas
odds_df = odds_df[new_cols]

odds_df.to_excel("output_corners_NNNN_AH.xlsx", index=False)

# Exibindo o DataFrame de odds
print(odds_df.shape)


(18, 27)
